# Reproducing Research Paper Results: TreeSwift

> Moshiri N (2020). "TreeSwift: a massively scalable Python package for trees." *SoftwareX*. 11:100436. [doi:10.1016/j.softx.2020.100436](https://doi.org/10.1016/j.softx.2020.100436)

# Introduction

In the world of biology, [phylogenetic trees](https://en.wikipedia.org/wiki/Phylogenetic_tree) can be used to study the evolution of organisms, and they are applicable to a wide range of disciplines, such as [epidemiology](https://en.wikipedia.org/wiki/Epidemiology) and [metagenomics](https://en.wikipedia.org/wiki/Metagenomics). During the COVID-19 pandemic, phylogenetic tools like [Nextstrain](https://nextstrain.org/ncov/gisaid) [(Hadfield *et al.*, 2018)](https://doi.org/10.1093/bioinformatics/bty407) were critical for tracking the spread and evolution of SARS-CoV-2.

When researchers build data analysis tools such as those created to study phylogenetic trees, programmers typically don't write the tools completely from scratch: instead, they typically build their tools upon on existing [packages](https://docs.python.org/3/tutorial/modules.html#packages), which are collections of code that perform common fundamental tasks in a given discipline. As a result, the performance of a data analysis tool (e.g. runtime, peak memory usage, etc.) doesn't just depend on the *tool developer's* code: it *also* depends on the code of the *package(s)* the developer *uses* in the tool.

As the cost of DNA sequencing continues to fall rapidly, viral and bacterial genome sequences are collected, resulting in extremely large phylogenetic trees. For example, the [global phylogenetic tree of SARS-CoV-2 genomes](https://taxonium.org/?backend=https://api.cov2tree.org) contains roughly 7.5 million leaves. Unfortunately, trees of this size are typically infeasible to analyze using existing tree packages in languages like Python.

[Moshiri (2020)](https://doi.org/10.1016/j.softx.2020.100436) introduces [TreeSwift](https://github.com/niemasd/TreeSwift), a user-friendly and massively scalable Python package for traversing and manipulating trees that is ideal for algorithms performed on ultra-large trees, and compares its performance against that of existing tools: [Biopython](https://biopython.org) (specifically the [Bio.Phylo](https://biopython.org/wiki/Phylo) module), [DendroPy](https://jeetsukumaran.github.io/DendroPy), and the [ETE Toolkit](http://etetoolkit.org/). In this activity, we will reproduce the key results of this manuscript by performing our own benchmark of these tools.

# Setting Up the Environment

Before we can run any analyses, we need to first install the packages we are benchmarking. We want to try to match the exact versions that were benchmarked in the paper to make sure we're reproducing the benchmark as accurately as possible: different versions may have slightly different performance (e.g. due to optimizations), which could slightly change the benchmarking results.

[Moshiri (2020)](https://doi.org/10.1016/j.softx.2020.100436) mentions that TreeSwift v1.1.3 was used (see [Code Metadata table](https://www.softxjournal.com/action/showFullTableHTML?isHtml=true&tableId=utbl1&pii=S2352-7110%2819%2930076-7)), but it does not specify version numbers for the other packages. However, it provides a link to the following GitHub repository that contains all scripts and data ([GitHub](https://github.com/) is a platform for developing and sharing code):

> https://github.com/niemasd/TreeSwift-Paper

This repository *also* does not specify version numbers for the other tools, but looking at the repository's [commit history](https://github.com/niemasd/TreeSwift-Paper/commits/master/) (essentially a history of modifications), we can see that the last [commit](https://docs.github.com/en/pull-requests/committing-changes-to-your-project/creating-and-editing-commits/about-commits) (essentially a modification) made before the paper's publication date (January 2020) was made on [July 30, 2019](https://github.com/niemasd/TreeSwift-Paper/commit/179f6b6a3c8de30dee305af8aff472c1636b1b7b). We can use this timestamp to estimate the version numbers for the packages that were used by assuming the most recent version of each package released before this date. The versions estimated for each of the tree packages are organized in the table below.

| Tool        | Version                                           |
| :---------: | :-----------------------------------------------: |
| TreeSwift   | [1.1.3](https://pypi.org/project/treeswift/1.1.3) |
| Biopython   | [1.74](https://pypi.org/project/biopython/1.74)   |
| DendroPy    | [4.4.0](https://pypi.org/project/DendroPy/4.4.0)  |
| ETE Toolkit | [3.1.1](https://pypi.org/project/ete3/3.1.1)      |

As you might have noticed, the version numbers in the table above are hyperlinks to specific pages on [PyPI](https://pypi.org/), which is the main repository for Python packages. We can use [`pip`](https://packaging.python.org/en/latest/key_projects/#pip), which is the most popular tool for installing Python packages from PyPI, to install all of the packages listed above by running the following cell, which should take around 1 minute to finish running.

In [ ]:
# install tree packages using pip
!pip install -q treeswift==1.1.3
!pip install -q biopython==1.74
!pip install -q dendropy==4.4.0
!pip install -q ete3==3.1.1

# Downloading the Data and Code

[Moshiri (2020)](https://doi.org/10.1016/j.softx.2020.100436) performs a benchmark experiment using data and code that were provided in the aforementioned GitHub repository:

> https://github.com/niemasd/TreeSwift-Paper

We can [clone](https://docs.github.com/en/repositories/creating-and-managing-repositories/cloning-a-repository) this repository to download all of its contents (namely the paper's data and analysis scripts) in order to perform the benchmark experiment by running the following cell, which should only take a few seconds to run.

In [2]:
!git clone https://github.com/niemasd/TreeSwift-Paper.git

Cloning into 'TreeSwift-Paper'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 220 (delta 0), reused 0 (delta 0), pack-reused 217 (from 1)
Receiving objects: 100% (220/220), 14.01 MiB | 32.09 MiB/s, done.
Resolving deltas: 100% (87/87), done.


After running the cell above, if you open the "Files" tab on Google Colab (the icon in the left navigation bar that looks like a folder), you will see that we now have a folder called `TreeSwift-Paper`, which contains the cloned contents of the aforementioned GitHub repository.

# Installing Analysis Dependencies

If you look through the Python scripts within the [`scripts`](https://github.com/niemasd/TreeSwift-Paper/tree/master/scripts) folder of the aforementioned GitHub repository, you'll notice that they [depend on](https://github.com/niemasd/TreeSwift-Paper/blob/ca754155014cd116ce48fdc570b5b47f9bac4cd0/scripts/figures.py#L16-L20) the following Python packages that are not part of the [Python Standard Library](https://docs.python.org/3/library/index.html) and must thus be installed: [NumPy](https://numpy.org/), [Matplotlib](https://matplotlib.org/), and [seaborn](https://seaborn.pydata.org/).

These packages are simply used to visualize the results of the benchmark experiment (i.e., they are not integral to the benchmark experiment itself), so it isn't critical for us to use the exact same versions that were used in the original paper. If we *really* wanted to, we could perform the same exploration as we did with the tree packages to find the most recent version of these packages released before [July 30, 2019](https://github.com/niemasd/TreeSwift-Paper/commit/179f6b6a3c8de30dee305af8aff472c1636b1b7b), but for the sake of simplicity, we will use the most recent versions of each of these packages.

We can install all of the packages needed for visualizing the results of our experiment by running the following cell, which should take around 30 seconds to finish running.

In [6]:
# install packages needed for visualizing our results
!pip install -q numpy
!pip install -q matplotlib
!pip install -q seaborn

# Exploring the Analysis Scripts and Data

Taking a closer look at the analysis scripts in the [`scripts`](https://github.com/niemasd/TreeSwift-Paper/tree/master/scripts) folder of the aforementioned GitHub repository, we can see that there are two scripts:

* [`time.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/time.py) — This script actually performs each individual task in the benchmark
    * There are individual functions for performing the tasks described at the beginning of [Section 3: Benchmarking](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#secd1e265) and [Figure 1](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#gr1) of the paper
    * **STOP and Think:** Read through the functions defined in `time.py`, and map them to each of the panels and curves in Figure 1 of the paper
* [`figures.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/figures.py) — This script takes as input the overall benchmarking results, and it produces the plots shown in [Figure 1](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#gr1) of the paper
    * **STOP and Think:** Many of the lines of this script deal with the *aesthetics* of the plot. Why do you think some of these aesthetic choices (e.g. plotting in log-scale, choosing different linestyles for the curves, etc.) were chosen?

The GitHub repository also provides the trees that were used to benchmark the tools, as well as the raw results of the benchmarking experiment, within the [`data`](https://github.com/niemasd/TreeSwift-Paper/tree/master/data) folder:

* `tree_nX.tre.gz` — These are the ([gzip](https://en.wikipedia.org/wiki/Gzip)-compressed) binary trees that were used to perform the benchmark, where `X` denotes the number of leaves
    * [`tree_n100.tre.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/tree_n100.tre.gz) is a binary tree with 100 leaves
    * [`tree_n1000.tre.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/tree_n1000.tre.gz) is a binary tree with 1,000 leaves
    * Etc.
* [`data.pkl.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/data.pkl.gz) — This is a ([gzip](https://en.wikipedia.org/wiki/Gzip)-compressed) [Pickle](https://docs.python.org/3/library/pickle.html) file containing the raw results of the benchmarking experiment
    * As you might have noticed when reviewing the scripts in the GitHub repository, this is the input to the [`figures.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/figures.py) script

After reviewing all of this, we can imagine the following steps for conducting our benchmark experiment:

1. Use [`time.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/time.py) to benchmark each tool on each tree file
2. Organize the results of [`time.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/time.py) into a [Pickle](https://docs.python.org/3/library/pickle.html) file structured in the same manner as [`data.pkl.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/data.pkl.gz)
3. Use [`figures.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/figures.py) on our own `data.pkl.gz` file to produce our own figures

## Exploring `data.pkl.gz`

As an initial "Step 0," since we ultimately want to create our own `data.pkl.gz` file, let's explore the one that was included in the aforementioned GitHub repository and was used to produce the figures in the paper. Let's load the provided [`data.pkl.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/data.pkl.gz) file and [pretty-print](https://en.wikipedia.org/wiki/Prettyprint) it so that we can better understand its structure. This cell should only take a couple of seconds to finish running.

In [7]:
# imports
import gzip
import pickle
import pprint

# open `data.pkl.gz` as a gzip file using `gzip.open`, and load the data using `pickle.load`
with gzip.open('TreeSwift-Paper/data/data.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# pretty-print the loaded data using `pprint`
# (`compact=True` puts more output on each line to make the long output more compact)
# (`width=150` increases how many characters per line of output; default = 80)
pprint.pp(data, compact=True, width=150)

{'postorder': {100: {'biophylo': [0.0017402172088623, 0.00176334381103516, 0.00175738334655762, 0.00218605995178223, 0.00174164772033691,
                                  0.00181150436401367, 0.0016939640045166, 0.00182294845581055, 0.00472354888916016, 0.00170350074768066],
                     'treeswift': [0.000358343124389648, 0.000356912612915039, 0.000364065170288086, 0.000354290008544922, 0.000373363494873047,
                                   0.000339269638061523, 0.00034332275390625, 0.000463008880615234, 0.000505924224853516, 0.000344276428222656],
                     'ete3': [0.000722169876098633, 0.000735282897949219, 0.000721216201782227, 0.000737190246582031, 0.000802040100097656,
                              0.000767946243286133, 0.000733137130737305, 0.0027778148651123, 0.000748872756958008, 0.000766515731811523],
                     'dendropy': [0.000357627868652344, 0.000357389450073242, 0.000374317169189453, 0.000366687774658203, 0.0003662109375,
               

As can be seen, [`data.pkl.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/data.pkl.gz) contains a Python [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) structured as follows:

* The keys of `data` are the [benchmarking tasks in `time.py`](https://github.com/niemasd/TreeSwift-Paper/blob/ca754155014cd116ce48fdc570b5b47f9bac4cd0/scripts/time.py#L348-L360)
* The keys of `data[task]` are the number of leaves of the trees (i.e., the `X` values in the files `tree_nX.tre.gz`)
* The keys of `data[task][X]` are the Python tool that were benchmarked
* The value associated with `data[task][X][tool]` is a Python [list](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists) containing all benchmarking measurements when executing `task` on `tree_nX.tre.gz` using `tool`
    * You may be wondering why there are multiple measurements for each benchmark
    * When performing benchmark experiments, [stochasticity](https://en.wikipedia.org/wiki/Stochastic) (i.e., randomness) in the system will cause some amount of deviation in measurements purely by chance, so we typically run the same benchmark task multiple times (i.e., we have multiple "technical replicates") to get a better estimate of the true measurement
    * From [Section 3: Benchmarking](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#secd1e265) of the paper: *Each point shows the average and 95% confidence interval **over 10 runs**.*
    * In other words, the 10 values contained within `data[task][X][tool]` are the measurements of the 10 technical replicates of running `task` on `tree_nX.tre.gz` using `tool`

## Reproducing the Original Plots

In addition to simply printing the contents of the original [`data.pkl.gz`](https://github.com/niemasd/TreeSwift-Paper/blob/master/data/data.pkl.gz) file that was provided in the aforementioned GitHub repository, let's also run the [`figures.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/figures.py) on it to reproduce the original benchmarking figures. The figures we ultimately produce from our own benchmarking experiment should (hopefully) match these figures reasonably well. This cell should take around 30 seconds to finish running.

In [5]:
!python3 TreeSwift-Paper/scripts/figures.py TreeSwift-Paper/data/data.pkl.gz

Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)


After running the cell above, if you open the "Files" tab on Google Colab (the icon in the left navigation bar that looks like a folder), you will see all of the PDFs produced by [`figures.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/figures.py).

*Note: You may need to close and reopen the "Files" tab to get the PDFs to appear after running the cell above.*

**STOP and Think:** Download the PDFs we just produced, make sure they match the plots in [Figure 1](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#gr1) of the paper.

Once you have matched each PDF with its corresponding plot in the paper, run the following cell to delete all of the PDFs to avoid confusion when we reproduce them with our own benchmarking results later. This cell should only take a second to run.

*Note: You may need to close and reopen the "Files" tab to get the PDFs to disappear after running the cell below.*

In [1]:
import glob
import os
for pdf in glob.glob('*.pdf'):
    os.remove(pdf)

# Running Our Own Benchmark

Now that we've explored the scripts and data associated with the paper, we're ready to run our own benchmark experiment! Let's first set up some definitions to help us run the exact measurements that were in the paper. This cell should only take a second to run.

In [8]:
# this patch makes the version of DendroPy we're using work in Python 3.10
#!sed -i '2i import collections; from collections.abc import MutableMapping; collections.MutableMapping = MutableMapping' TreeSwift-Paper/scripts/time.py

# we want 10 replicates per measurement
NUM_REPS = 10

# set the "number of leaves" values we want to run for each task based on Figure 1 of the paper
# we're only going to go up to 10,000 leaves for the sake of time, but feel free to go all the way up to 1,000,000 if you'd like by modifying the lists below
NUM_LEAVES = {
    'memory':          [100, 1000, 10000],
    'load_tree':       [100, 1000, 10000],
    'preorder':        [100, 1000, 10000],
    'postorder':       [100, 1000, 10000],
    'inorder':         [100, 1000, 10000],
    'levelorder':      [100, 1000, 10000],
    'mrca':            [100, 1000, 10000],
    'distance_matrix': [100, 1000],
}

# some tools are particularly inefficient or lack certain functionality, so let's use the same max "number of leaves" for each tool from Figure 1 of the paper
MAX_NUM_LEAVES = {
    'memory': {
        'dendropy': 100000,
    },
    'inorder': {
        'biophylo': 0,
        'ete3':     0,
    },
    'mrca': {
        'dendropy': 10000,
        'biophylo': 10000,
        'ete3':     10000,
    }
}

Before we actually start running the benchmark experiment, note that, even though we're using the same versions of TreeSwift, DendroPy, Biopython, and ETE as were used in the original paper, we still expect deviations in our measurements due to differences in the compute environments we're running them on. Specifically, from the second paragraph in [Section 3: Benchmarking](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#secd1e265) of the paper:

> *Timing was performed on a computer running CentOS release 6.6 (Final) with an Intel(R) Xeon(R) CPU E5-2670 0 at 2.60 GHz and 32 GB of RAM.*

Let's take a look at the specifications of the environment this notebook is running in by running the cell below, which should only take a second to run.

In [9]:
# print details about the operating system
print("=== Operating System ===")
!cat /etc/os-release
print()

# print details about the RAM
print("=== RAM ===")
!free --mega
print()

# print details about the CPU
print("=== CPU ===")
!cat /proc/cpuinfo
print()

=== Operating System ===
cat: /etc/os-release: No such file or directory

=== RAM ===
/bin/bash: free: command not found

=== CPU ===
cat: /proc/cpuinfo: No such file or directory



**STOP and Think:** Read through the Operating System, RAM, and CPU summaries printed above. How do they compare against the specifications of the computer used in the original paper? How do we expect our measurements to differ from the ones in the paper based on these differences?

Now that we've thought about this carefully, let's actually run our benchmark experiment!

**WARNING:** The following cell will take a *long* time to run (**probably close to 30 minutes**)!!! You can adjust the `NUM_REPS`, `NUM_LEAVES`, and `MAX_NUM_LEAVES` variables we defined at the beginning of this section if desired (e.g. to run fewer than 10 replicates per measurement, to restrict the experiment to just trees with up to 1,000 leaves, etc.), but note that this will impact the results of your experiment.

In [10]:
# run the benchmark experiment
import subprocess
data = dict()
for task in NUM_LEAVES:
    data[task] = dict()
    print("=== Running task: %s ===" % task)
    for X in NUM_LEAVES[task]:
        data[task][X] = dict()
        print("- X = %d leaves." % X, end='')
        for tool in ['dendropy', 'treeswift', 'ete3']:
            if task in MAX_NUM_LEAVES and tool in MAX_NUM_LEAVES[task] and X > MAX_NUM_LEAVES[task][tool]:
                continue # skip this tool if X > max num leaves we want to run
            data[task][X][tool] = list()
            print(" %s" % tool, end='')
            command = ['python3', 'TreeSwift-Paper/scripts/time.py', 'TreeSwift-Paper/data/tree_n%d.tre.gz' % X, tool, task]
            for replicate in range(1, NUM_REPS + 1):
                print('.', end='')
                print(command)
                measurement = subprocess.check_output(command).decode().strip()
                if measurement != 'NA':
                    data[task][X][tool].append(float(measurement))
        print()

=== Running task: memory ===
- X = 100 leaves. dendropy.['python3', 'TreeSwift-Paper/scripts/time.py', 'TreeSwift-Paper/data/tree_n100.tre.gz', 'dendropy', 'memory']


Traceback (most recent call last):
  File "/Users/brandonwerbel/Documents/sandbox/treeswift-env/TreeSwift-Paper/scripts/time.py", line 2, in <module>
    from Bio import Phylo
ModuleNotFoundError: No module named 'Bio'


CalledProcessError: Command '['python3', 'TreeSwift-Paper/scripts/time.py', 'TreeSwift-Paper/data/tree_n100.tre.gz', 'dendropy', 'memory']' returned non-zero exit status 1.

Now that we've finished running the benchmark experiment, we can save the raw results in a file called `data.pkl.gz`. This cell should only take a second to run.

In [11]:
# save `data` to a file called `data.pkl.gz`
import gzip
import pickle
with gzip.open('data.pkl.gz', 'wb') as f:
    pickle.dump(data, f)

Now, let's create plots from our own benchmarking results! This cell should take around 30 seconds to finish running.

In [12]:
!python3 TreeSwift-Paper/scripts/figures.py data.pkl.gz

{'memory': {100: {'dendropy': []}}}
Traceback (most recent call last):
  File "/Users/brandonwerbel/Documents/sandbox/treeswift-env/TreeSwift-Paper/scripts/figures.py", line 18, in <module>
    from matplotlib import rcParams
ModuleNotFoundError: No module named 'matplotlib'


After running the cell above, if you open the "Files" tab on Google Colab (the icon in the left navigation bar that looks like a folder), you will see all of the PDFs produced by [`figures.py`](https://github.com/niemasd/TreeSwift-Paper/blob/master/scripts/figures.py).

*Note: You may need to close and reopen the "Files" tab to get the PDFs to appear after running the cell above.*

**STOP and Think:** Download the PDFs we just produced, compare them against the plots in [Figure 1](https://www.softxjournal.com/article/S2352-7110(19)30076-7/fulltext#gr1) of the paper.